In [1]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dropout, Conv2D, Flatten, MaxPooling2D, Dense, LSTM
import numpy as np

In [2]:
def build_model(drop=0.5, optimizer = 'adam'):
    inputs = Input(shape= (28*28, 1 ), name = 'input')
    x = LSTM(51, activation = 'relu', name = 'hidden1')(inputs)
    x = Dropout(drop)(x)
    x = Dense(256, activation = 'relu', name = 'hidden2')(x)
    x = Dropout(drop)(x)
    x = Dense(128, activation = 'relu', name = 'hidden3')(x)
    x = Dropout(drop)(x)
    outputs = Dense(10, activation = 'softmax', name = 'output')(x)
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(optimizer = optimizer, metrics = ['acc'],
                  loss = 'categorical_crossentropy')
    return model

In [5]:
# parameter
def create_hyperparameters(): # epochs, node, acivation 추가 가능
    batches = [64, 128, 256]
    optimizers = ['rmsprop', 'adam', 'adadelta']
    dropout = np.linspace(0.1, 0.5, 5).tolist()                           
    return {'batch_size' : batches, 'optimizer': optimizers, 
           'drop': dropout}

hyperparameters = create_hyperparameters()

In [6]:
# wrapper
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier          
model = KerasClassifier(build_fn = build_model, verbose = 1)

In [8]:
# gridsearch
from sklearn.model_selection import GridSearchCV
search = GridSearchCV(model, hyperparameters, cv = 3) # cv = cross_validation
# batches * optimizers * dropout * cv
#   3     *     3      *    5    *  3 = 225번의 model이 돌아감

In [ ]:
'''
GridSearch 는 우리가 지정해준 몇 가지 잠재적
Parameter들의 후보군들의 조합 중에서 가장 Best 조합을 찾아줍니다.
어떻게 보면 우리가 하나하나 대입해 가면서 loss를 확인하는 작업을
GridSearch는 대신 해준다고 보면 됩니다.
또한, sklearn 패키지에서 제공해주고 있기때문에 매우 손쉽게 사용할 수 있습니다.

하지만, 가장 큰 단점은 우리가 지정해 준 hyperparameter 후보군의 갯수만큼
비례하여 시간이 늘어기 때문에 최적의 조합을 찾을 때까지
시간이 매우 오래 걸린다는 단점이 있습니다.
'''

In [7]:
# randomsearch
from sklearn.model_selection import RandomizedSearchCV         
search = RandomizedSearchCV(model, hyperparameters, cv = 3) #, n_jobs=5
# n_jobs = 사용할 core의 수를 정해줌
# batches * optimizers * dropout * cv
#   3     *     3      *    5    *  3 = 225번의 model이 돌아감

In [ ]:
'''
gridsearch의 단점을 피하기 위해 나온 방법
난수를 이용하여 매개변수 조합을 생성
-> gridsearch는 매개변수가 될 수들을 미리 정하지만, random search는 난수를 사용
이 때 각각의 매개변수 범위나 간격 등을 설정해야 함
보통 논문을 보면 제시하는 기본값을 중심으로 적절한 범위를 설정함
'''